# Linear predictors

In [1]:
import pickle
import numpy as np

In [2]:
with open('../pickles/test_cells.pkl', 'rb') as f:
    cells_te = pickle.load(f)

In [3]:
def LAD_cv_lin_cellselect(X, y, gene_comp, kf=10):
    kfold = KFold(n_splits=kf, shuffle=True, random_state=0)
    
    if len(gene_comp) > 0:
        X_in = np.copy(X[:, comp_ind])
    else:
        X_in = np.copy(X)
        
    w_list = []
    bias_list = []
    
    te = np.hstack([cells_te+(6078*i) for i in range(5)])
    cells_tv = np.sort(np.delete(range(6078), cells_te))
    tv = np.sort(np.hstack([cells_tv+(6078*i) for i in range(5)]))
    
    X_te = X_in[te]
    y_te = y[te]
    X_tv = X_in[tv]
    y_tv = y[tv]
    
    for (cell_tr, cell_v) in (kfold.split(range(5470))):
        v = np.hstack([cell_v+(5470*i) for i in range(5)])
        tr = np.delete(range(27350), v)
        X_tr, y_tr = X_tv[tr], y_tv[tr]
        X_v, y_v = X_tv[v], y_tv[v]
        
        w,bias = infer_LAD_v(X_tr, y_tr, X_v, y_v)
        w_list.append(w)
        bias_list.append(bias)
        
    w = np.mean(w_list, axis=0)
    bias = np.mean(bias_list, axis=0)
    
    print(w.shape, bias.shape)
    if y.shape[1] == 27:
        X_init = np.copy(X[:, comp_ind][te])
    else:
        X_init = np.copy(X[te])
    
    yp = X_init + bias + X_te.dot(w)
    yp_neg = np.copy(yp)
    yp[yp<0] = 0
    ya = X_init + y_te
    ferror = np.sum(np.abs(yp - ya), axis=0)/np.sum(np.abs(ya), axis=0)
    ferror_neg = np.sum(np.abs(yp_neg - ya), axis=0)/np.sum(np.abs(ya), axis=0)
    error1 = np.abs(yp - ya)
    
    dic = {
        'yp_neg': yp_neg,
        'yp': yp,
        'ferror': ferror,
        'ferror_neg': ferror_neg,
        'error_cell_l1': error1,
        'w': w,
        'bias': bias
    }
    return dic

def infer_all_LAD(X_all, y_all, y_comp):
    res=[]
    results = LAD_cv_lin_cellselect(X_all, y_all, gene_comp=[], kf=10)
    res.append(results)
#     results = LAD_cv_lin_cellselect(X_all, y_comp, gene_comp=[], kf=10)
#     res.append(results)
#     results = LAD_cv_lin_cellselect(X_all, y_comp, gene_comp=comp_ind, kf=10)
#     res.append(results)
    return res

In [4]:
data_all = np.loadtxt('../data/data_complete_tf_new.txt')

all_bin = np.vsplit(data_all, 6)
all_init = np.vstack([all_bin[i] for i in range(5)])
all_diff = np.vstack([all_bin[i+1]-all_bin[i] for i in range(5)])

complete_all = ([int(x) - 1 for x in open('../data/indices_complete.txt','r').readline().split()])
comp_ind = list(map(int, list((np.array(complete_all)[::6]-3)/6)))

data_comp = np.copy(data_all[:, comp_ind])
comp_bin = np.vsplit(data_comp, 6)
comp_diff = np.vstack([comp_bin[i+1] - comp_bin[i] for i in range(5)])

In [5]:
# PATH = '../pickles/'

# LAD_lin_cellselect = infer_all_LAD(all_init, all_diff, comp_diff)

# with open(PATH + 'LAD_lin_tvt.pkl', 'wb') as f:
#     pickle.dump(LAD_lin_cellselect, f)

# Linear + quadratic predictors
The computations were carried out using swarm. The materials are in ./HPC/2. Inference - lin+quad/